# Data Cleaning
- only keeping movies that were released (dropping anything pre-release)
- get rid of the columns that we wont use
- create binary response 

In [1]:
import pandas as pd
import numpy as np

In [83]:
# importing csv file
movies_metadata = pd.read_csv('movies_metadata.csv')

C:\Users\csky2\AppData\Local\Temp\ipykernel_5024\1536947796.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('movies_metadata.csv')


In [84]:
# only keeping movies that were released (dropping anything pre-release)
# drops about 400 movies
movies_metadata = movies_metadata.drop(np.where(movies_metadata['status'] != 'Released')[0])

In [85]:
# dropping 9 columns that we will not use
movies_metadata = movies_metadata.drop(['adult', 'homepage', 'id', 'imdb_id', 'original_title', 'poster_path', 'status', 'video', 'overview'], axis=1)

In [86]:
# cleaning belongs_to_collection - making it binary
movies_metadata.loc[~movies_metadata.belongs_to_collection.isnull(), 'belongs_to_collection'] = 1  # not nan
movies_metadata.loc[movies_metadata.belongs_to_collection.isnull(), 'belongs_to_collection'] = 0

In [87]:
movies_metadata.columns

Index(['belongs_to_collection', 'budget', 'genres', 'original_language',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'tagline',
       'title', 'vote_average', 'vote_count'],
      dtype='object')

In [88]:
print('num of movies that made money:', movies_metadata[movies_metadata['revenue']!=0].shape[0])
print('num of movies that didnt make money:', movies_metadata[movies_metadata['revenue']==0].shape[0])

num of movies that made money: 7395
num of movies that didnt make money: 37619


In [165]:
# making y binary
movies_classification = movies_metadata.copy()
movies_classification.loc[movies_metadata['revenue']!=0, 'revenue'] = 1
movies_classification.loc[movies_metadata['revenue']==0, 'revenue'] = 0

In [166]:
movies_classification.genres[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [158]:
# turning genres into dummy vars

# replace all the [] with ''
# make each obs a list - string.split(",")
# blank dict genre_types
# for each list in movies_classification.genres:
#     for each dict in the list:
#          if id not in genre_types:
#             genre_types |= dict

# for name in genre_types:
#     add a column to movies_classification
# make all obs vals 0

# for each name in dict:
#    if name is in obs:
#       movies_classification.loc[row, name] = 1
# do movies_classification.apply(lambda...

In [170]:
movies_classification.genres = movies_classification.genres.apply(lambda x:eval(x))

In [199]:
genre_types = []
for list_of_dicts in movies_classification.genres:
    for dictionary in list_of_dicts:
        if dictionary['name'] not in genre_types:
            genre_types.append(dictionary['name'])

In [203]:
for genre in genre_types:
    movies_classification[genre] = 0


SyntaxError: invalid syntax (3198499710.py, line 3)

In [206]:
movies_classification.genres.isin(['Horror']).sum()

0

In [204]:
    movies_classification['Horror'].apply(lambda x:1 if movies_classification.genres.isin(['Horror']))

SyntaxError: invalid syntax (1088092745.py, line 1)

In [38]:
dummies = pd.get_dummies(movies_classification)

C:\Users\csky2\AppData\Local\Temp\ipykernel_5024\1535929241.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  dummies = pd.get_dummies(movies_classification)


In [39]:
dummies.shape

(45014, 155331)